# Episodic Memory and Generative Completion - Full Pipeline

1. Train VQ-VAE Model
2. Apply the trained model on entire images and store codebook as quantizers, latent spaces as indices, and then reconstruct images
3. Train a classifier model
4. Train a transformer, here distilbert
5. Ensemble all together to see the performance of transformer on different type of masking

> Set up essentials

Including:
1. modules and libraries
2. functions
3. config data
4. etc.

## Step 1 - Train VQVAE Model 

- Obtain best model
- Obtain best hyperparameters
- Record Test Error

In [ ]:
import os, torch, yaml, optuna, mlflow, subprocess, json
from torch import nn, optim
from vqvae import FlatVQVAE
from utils import CustomImageNetDataV2, CustomLoader
from torch import distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler

In [4]:
# Parse config file
with open('config.yaml', 'r') as f:
    config_data = yaml.full_load(f)

img_train_dir = config_data['path']['image_net_train']
img_test_dir = config_data['path']['image_net_test']
img_val_dir = config_data['path']['image_net_val']
model_path = config_data['path']['vqvae_model']

In [5]:
def get_available_gpus(threshold_percent=20):
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=memory.used,memory.total', '--format=csv,nounits,noheader'],
                                stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        lines = result.stdout.strip().split('\n')
        usage = [(i, int(u.split(',')[0]) / int(u.split(',')[1]) * 100) for i, u in enumerate(lines)]
        available = [(i, round(percent, 2)) for i, percent in usage if percent < threshold_percent]
        print(f"GPUs below {threshold_percent}% usage:")
        for i, p in available:
            print(f"GPU {i}: {p}% used")
        return [i for i, _ in available]
    except Exception as e:
        print(f"Error checking GPU status: {e}")
        return []

# Example usage
available_gpu_ids = get_available_gpus(threshold_percent=30)

# Manual selection (for now, just pick the first one if available)
selected_gpu_ids = available_gpu_ids if available_gpu_ids else [0]
device = torch.device(f"cuda:{selected_gpu_ids[0]}" if torch.cuda.is_available() else "cpu")


GPUs below 30% usage:
GPU 0: 0.15% used


In [ ]:
def setup_distributed():
    '''
    Useful to integrate in notebook
    '''
    if torch.cuda.device_count() > 1:
        dist.init_process_group(backend="nccl")
        local_rank = int(os.environ.get("LOCAL_RANK", 0))
        torch.cuda.set_device(local_rank)
        print(f"Distributed initialized. Using GPU {local_rank}")
        return True
    return False

def get_loader(dataset, batch_size, shuffle, distributed):
    if distributed:
        return CustomLoader(dataset, batch_size=batch_size, shuffle = shuffle, distributed = distributed).data_loader
    else:
        return CustomLoader(dataset, batch_size=batch_size, shuffle = shuffle).data_loader

> Load datasets

In [5]:
# 📂 Load datasets
train_set = CustomImageNetDataV2(image_dir = img_train_dir, image_type = 'original', folder_label = 'int_id')
val_set = CustomImageNetDataV2(image_dir = img_val_dir, image_type = 'original', folder_label = 'int_id')
test_set = CustomImageNetDataV2(image_dir = img_test_dir, image_type = 'original', folder_label = 'int_id')

> Use Optuna to find best hyperparameters and log them via MLflow

In [ ]:
# 🧠 Optuna objective function
def objective(trial):
    # 🔧 Hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True) # samples values logarithmically, favoring smaller values
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-2, log=True) # samples values logarithmically, favoring smaller values
    latent_loss_weight = trial.suggest_float("latent_loss_weight", 0.1, 2.0, log=True) # use logarithmic sampling instead of linear increments since parameter behaves non-linearly — small changes at low values have big effects, but want to explore larger ones as well
    diversity_loss_weight = trial.suggest_float("diversity_loss_weight", 0.0, 1.0) # samples linearly without discrimination between values in the specified range
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128]) # picks one from the list
    num_epochs = trial.suggest_int('num_epochs', 10, 50) # picks an integer, but doesn’t “try all” in sequence.

    distributed = dist.is_available() and dist.is_initialized()
    shuffle = not distributed

    train_loader = get_loader(train_set, batch_size, shuffle, distributed)
    val_loader = get_loader(val_set, batch_size, False, distributed)

    # 🧠 Model
    model = FlatVQVAE().to(device)
    if torch.cuda.device_count() > 1:
        model = torch.nn.parallel.DataParallel(model)
    # if distributed:
    #     model = DDP(model, device_ids=[device]) # useful for .py cript execute by torchrun "torchrun --nproc_per_node=4 train.py" or python -m torch.distributed.launch

    recon_criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3)

    mlflow.start_run(run_name=f"Optuna_Trial_{trial.number}")
    mlflow.log_params(trial.params)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for inputs, _ in train_loader:
            inputs = inputs.to(device)
            optimizer.zero_grad()
            recon, latent_loss, diversity_loss, _ = model(inputs)
            recon_loss = recon_criterion(recon, inputs)
            loss = recon_loss + latent_loss_weight * latent_loss + diversity_loss_weight * diversity_loss
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)

        train_loss /= len(train_loader.dataset)

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, _ in val_loader:
                inputs = inputs.to(device)
                recon, latent_loss, diversity_loss, _ = model(inputs)
                recon_loss = recon_criterion(recon, inputs)
                loss = recon_loss + latent_loss_weight * latent_loss + diversity_loss_weight * diversity_loss
                val_loss += loss.item() * inputs.size(0)

        val_loss /= len(val_loader.dataset)
        scheduler.step(val_loss)

        mlflow.log_metric("train_loss", train_loss, step=epoch)
        mlflow.log_metric("val_loss", val_loss, step=epoch)
        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    # Save model
    torch.save(model.state_dict(), f"{model_path}/best_vqvae_model_trial_{trial.number}.pth")
    mlflow.pytorch.log_model(model, artifact_path="vqvae_model")

    return val_loss

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

best_trial = study.best_trial
print("Best Trial:")
for k, v in best_trial.params.items():
    print(f"{k}: {v}")
with open(f"{model_path}/best_vqvae_params_trial_{best_trial.number}.json", 'w') as f:
    json.dump(best_trial.params, f, indent=4)

# Reload best model
model = FlatVQVAE().to(device)
model.load_state_dict(torch.load(f"{model_path}/best_vqvae_model_trial_{best_trial.number}.pth"))
model.eval()

# Test loader
test_loader = get_loader(test_set, batch_size=best_trial.params['batch_size'], shuffle=False, distributed=False)

# Evaluate
test_loss = 0.0
with torch.no_grad():
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        recon, latent_loss, diversity_loss, _ = model(inputs)
        recon_loss = nn.MSELoss()(recon, inputs)
        loss = recon_loss + best_trial.params['latent_loss_weight'] * latent_loss + best_trial.params['diversity_loss_weight'] * diversity_loss
        test_loss += loss.item() * inputs.size(0)

test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}")


[I 2025-09-24 09:54:00,252] A new study created in memory with name: no-name-6c8a42dd-ae53-4938-a2f9-8c8a98a77cb4
/home/mohsen/anaconda3/lib/python3.11/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv2d(input, weight, bias, self.stride,


# Step 2 - Reconstruct Images

- Apply trained model on test data to 